# Production Deployment Guide
## Enterprise Deployment and Scaling - Notebook 7

**Objective**: Guide through production deployment of the economic forecasting multi-agent system with enterprise-grade considerations.

### What You'll Learn:
- Cloud deployment architecture and setup
- API configuration and security
- Scaling considerations and performance optimization
- Monitoring, logging, and alerting
- Cost optimization and maintenance

## 1. Production Architecture Overview

In [ ]:
# Import required libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import asyncio
import warnings
import yaml
import json
from datetime import datetime
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append('../src')

# Import production components
from agents.team_coordinator import EconomicTeamCoordinator
from google.adk.models.google_llm import Gemini
from google.genai import types

# Setup visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Production deployment components imported successfully")

## 2. Production Configuration

In [ ]:
# Create production configuration
print("⚙️ Creating Production Configuration...")

production_config = {
    "deployment": {
        "environment": "production",
        "version": "1.0.0",
        "deployment_date": datetime.now().strftime('%Y-%m-%d'),
        "maintainer": "Economic Forecasting Team"
    },
    "api_config": {
        "bea_api": {
            "base_url": "https://apps.bea.gov/api/data",
            "timeout": 30,
            "retry_attempts": 5,
            "rate_limit": "1000 requests per hour"
        },
        "gemini_api": {
            "model": "gemini-2.0-flash-exp",
            "max_tokens": 8192,
            "temperature": 0.1,
            "timeout": 60
        }
    },
    "performance": {
        "max_concurrent_sessions": 100,
        "session_timeout_minutes": 60,
        "cache_ttl_hours": 24,
        "max_memory_usage_mb": 1024
    },
    "monitoring": {
        "log_level": "INFO",
        "metrics_interval_seconds": 60,
        "alert_thresholds": {
            "error_rate": 0.05,
            "response_time_ms": 5000,
            "memory_usage": 0.8
        }
    },
    "security": {
        "api_key_rotation_days": 90,
        "rate_limiting": True,
        "input_validation": True,
        "output_sanitization": True
    }
}

# Save configuration
with open('production_config.yaml', 'w') as f:
    yaml.dump(production_config, f, default_flow_style=False)

print("✅ Production configuration saved: production_config.yaml")
print("\n📋 Configuration Overview:")
print(f"   Environment: {production_config['deployment']['environment']}")
print(f"   Version: {production_config['deployment']['version']}")
print(f"   Max Sessions: {production_config['performance']['max_concurrent_sessions']}")
print(f"   Log Level: {production_config['monitoring']['log_level']}")

## 3. Cloud Deployment Architecture

In [ ]:
# Create deployment architecture visualization
print("🏗️ Creating Cloud Deployment Architecture...")

def create_architecture_diagram():
    """Create visualization of production architecture"""

    # Define components and their positions
    components = {
        'Load Balancer': {'x': 0, 'y': 0, 'color': '#1f77b4', 'shape': 'circle'},
        'API Gateway': {'x': 1, 'y': 0, 'color': '#ff7f0e', 'shape': 'diamond'},
        'Agent Service': {'x': 2, 'y': 1, 'color': '#2ca02c', 'shape': 'square'},
        'Data Service': {'x': 2, 'y': -1, 'color': '#d62728', 'shape': 'square'},
        'Database': {'x': 3, 'y': 0, 'color': '#9467bd', 'shape': 'triangle-up'},
        'Cache Layer': {'x': 4, 'y': 1, 'color': '#8c564b', 'shape': 'hexagon'},
        'Monitoring': {'x': 4, 'y': -1, 'color': '#e377c2', 'shape': 'pentagon'},
        'External APIs': {'x': 5, 'y': 0, 'color': '#7f7f7f', 'shape': 'star'}
    }

    fig = go.Figure()

    # Add component nodes
    for component, props in components.items():
        fig.add_trace(go.Scatter(
            x=[props['x']], y=[props['y']],
            mode='markers+text',
            marker=dict(
                size=25,
                color=props['color'],
                symbol=props['shape'],
                line=dict(width=2, color='DarkSlateGrey')
            ),
            text=component,
            textposition="middle center",
            name=component,
            hoverinfo='text'
        ))

    # Define connections
    connections = [
        ('Load Balancer', 'API Gateway'),
        ('API Gateway', 'Agent Service'),
        ('API Gateway', 'Data Service'),
        ('Agent Service', 'Database'),
        ('Data Service', 'Database'),
        ('Agent Service', 'Cache Layer'),
        ('Data Service', 'Cache Layer'),
        ('Agent Service', 'Monitoring'),
        ('Data Service', 'Monitoring'),
        ('Agent Service', 'External APIs'),
        ('Data Service', 'External APIs')
    ]

    # Add connection lines
    for start, end in connections:
        fig.add_trace(go.Scatter(
            x=[components[start]['x'], components[end]['x']],
            y=[components[start]['y'], components[end]['y']],
            mode='lines',
            line=dict(color='gray', width=2, dash='dot'),
            showlegend=False
        ))

    fig.update_layout(
        title='Production Deployment Architecture',
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        showlegend=False,
        height=500,
        plot_bgcolor='white'
    )

    return fig

# Create and display architecture diagram
architecture_fig = create_architecture_diagram()
architecture_fig.show()

print("✅ Cloud architecture diagram created")
print("\n🏗️ Architecture Components:")
print("   • Load Balancer - Distributes incoming traffic")
print("   • API Gateway - Handles API routing and security")
print("   • Agent Service - Multi-agent system core")
print("   • Data Service - Economic data management")
print("   • Database - Persistent session and state storage")
print("   • Cache Layer - Performance optimization")
print("   • Monitoring - Real-time system observability")
print("   • External APIs - BEA and Gemini API integration")

In [ ]:
# Create deployment automation scripts
print("🤖 Generating Deployment Automation Scripts...")

# Dockerfile for containerization
dockerfile_content = """
# Multi-stage build for optimized production image
FROM python:3.9-slim as builder

WORKDIR /app

# Install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

FROM python:3.9-slim as production

WORKDIR /app

# Copy installed packages
COPY --from=builder /usr/local/lib/python3.9/site-packages/ /usr/local/lib/python3.9/site-packages/
COPY --from=builder /usr/local/bin/ /usr/local/bin/

# Copy application code
COPY src/ ./src/
COPY config/ ./config/
COPY main.py .

# Create non-root user
RUN useradd --create-home --shell /bin/bash app
USER app

# Environment variables
ENV PYTHONPATH=/app/src
ENV PYTHONUNBUFFERED=1

# Health check
HEALTHCHECK --interval=30s --timeout=30s --start-period=5s --retries=3 \\
    CMD python -c "from src.agents.team_coordinator import EconomicTeamCoordinator; print('Health check passed')"

# Expose port
EXPOSE 8080

# Run application
CMD ["python", "main.py"]
"""

with open('Dockerfile', 'w') as f:
    f.write(dockerfile_content)

print("✅ Dockerfile created: Dockerfile")

# Docker Compose for local development
docker_compose_content = """
version: '3.8'

services:
  economic-forecasting:
    build: .
    ports:
      - "8080:8080"
    environment:
      - BEA_API_KEY=${BEA_API_KEY}
      - GOOGLE_API_KEY=${GOOGLE_API_KEY}
      - LOG_LEVEL=INFO
    volumes:
      - ./data:/app/data
      - ./logs:/app/logs
    healthcheck:
      test: ["CMD", "python", "-c", "from src.agents.team_coordinator import EconomicTeamCoordinator; print('ok')"]
      interval: 30s
      timeout: 10s
      retries: 3
    restart: unless-stopped

  redis:
    image: redis:7-alpine
    ports:
      - "6379:6379"
    volumes:
      - redis_data:/data
    restart: unless-stopped

  monitoring:
    image: prom/prometheus:latest
    ports:
      - "9090:9090"
    volumes:
      - ./monitoring/prometheus.yml:/etc/prometheus/prometheus.yml
    restart: unless-stopped

volumes:
  redis_data:

networks:
  default:
    name: economic-forecasting-network
"""

with open('docker-compose.yml', 'w') as f:
    f.write(docker_compose_content)

print("✅ Docker Compose file created: docker-compose.yml")

# Kubernetes deployment manifest
kubernetes_manifest = """
apiVersion: apps/v1
kind: Deployment
metadata:
  name: economic-forecasting
  labels:
    app: economic-forecasting
spec:
  replicas: 3
  selector:
    matchLabels:
      app: economic-forecasting
  template:
    metadata:
      labels:
        app: economic-forecasting
    spec:
      containers:
      - name: economic-forecasting
        image: economic-forecasting:latest
        ports:
        - containerPort: 8080
        env:
        - name: BEA_API_KEY
          valueFrom:
            secretKeyRef:
              name: api-secrets
              key: bea-api-key
        - name: GOOGLE_API_KEY
          valueFrom:
            secretKeyRef:
              name: api-secrets
              key: google-api-key
        resources:
          requests:
            memory: "512Mi"
            cpu: "250m"
          limits:
            memory: "1Gi"
            cpu: "500m"
        livenessProbe:
          httpGet:
            path: /health
            port: 8080
          initialDelaySeconds: 30
          periodSeconds: 10
        readinessProbe:
          httpGet:
            path: /ready
            port: 8080
          initialDelaySeconds: 5
          periodSeconds: 5

---
apiVersion: v1
kind: Service
metadata:
  name: economic-forecasting-service
spec:
  selector:
    app: economic-forecasting
  ports:
  - port: 80
    targetPort: 8080
  type: LoadBalancer
"""

with open('kubernetes-deployment.yaml', 'w') as f:
    f.write(kubernetes_manifest)

print("✅ Kubernetes deployment manifest created: kubernetes-deployment.yaml")

print("\n🚀 Deployment Automation Summary:")
print("   • Dockerfile - Container image definition")
print("   • docker-compose.yml - Local development stack")
print("   • kubernetes-deployment.yaml - Production K8s deployment")
print("   • Ready for cloud deployment on GCP, AWS, or Azure")

In [ ]:
# Performance optimization configurations
print("⚡ Configuring Performance Optimizations...")

optimization_config = {
    "caching": {
        "redis": {
            "host": "localhost",
            "port": 6379,
            "db": 0,
            "ttl": {
                "economic_data": 3600,  # 1 hour
                "session_data": 1800,   # 30 minutes
                "forecast_results": 7200  # 2 hours
            }
        }
    },
    "database": {
        "connection_pool": {
            "max_connections": 20,
            "max_overflow": 10,
            "pool_timeout": 30,
            "pool_recycle": 3600
        },
        "indexes": [
            "sessions_user_id",
            "sessions_created_time",
            "economic_data_date"
        ]
    },
    "api_optimization": {
        "batch_processing": True,
        "parallel_requests": 5,
        "request_timeout": 30,
        "retry_strategy": "exponential_backoff"
    },
    "memory_management": {
        "session_cleanup_interval": 300,  # 5 minutes
        "max_session_size_mb": 10,
        "garbage_collection_threshold": 0.8
    }
}

# Save optimization config
with open('optimization_config.json', 'w') as f:
    json.dump(optimization_config, f, indent=2)

print("✅ Performance optimization configuration saved: optimization_config.json")

# Create performance benchmarking script
benchmark_script = """
#!/usr/bin/env python3
"""Performance Benchmarking Script for Economic Forecasting System"""

import asyncio
import time
import pandas as pd
from datetime import datetime
from src.agents.team_coordinator import EconomicTeamCoordinator

class PerformanceBenchmark:
    def __init__(self, team_coordinator):
        self.team = team_coordinator
        self.results = []

    async def benchmark_agent(self, agent_name, test_query, iterations=10):
        """Benchmark individual agent performance"""
        times = []
        successes = 0

        for i in range(iterations):
            start_time = time.time()
            try:
                result = await self.team.run_complete_analysis(test_query)
                if result['status'] == 'success':
                    successes += 1
            except Exception as e:
                print(f"Error in iteration {i}: {e}")

            end_time = time.time()
            times.append(end_time - start_time)

        avg_time = sum(times) / len(times)
        success_rate = (successes / iterations) * 100

        return {
            'agent': agent_name,
            'average_time': avg_time,
            'success_rate': success_rate,
            'iterations': iterations
        }

    async def run_comprehensive_benchmark(self):
        """Run comprehensive performance benchmarks"""

        test_scenarios = [
            ('GDP Analysis', 'Analyze current GDP trends and provide forecast'),
            ('Economic Health', 'Assess overall economic health with key indicators'),
            ('Multi-Indicator', 'Analyze GDP, unemployment, and inflation together')
        ]

        print("🚀 Starting Comprehensive Performance Benchmark...")

        for scenario_name, query in test_scenarios:
            print(f"\\n📊 Benchmarking: {scenario_name}")
            result = await self.benchmark_agent(scenario_name, query, iterations=5)
            self.results.append(result)

            print(f"   Average Time: {result['average_time']:.2f}s")
            print(f"   Success Rate: {result['success_rate']:.1f}%")

        return self.results

    def generate_report(self):
        """Generate performance benchmark report"""

        if not self.results:
            return "No benchmark results available"

        df = pd.DataFrame(self.results)

        report = f"""
# Performance Benchmark Report
## Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Summary Statistics
- Total Scenarios Tested: {len(self.results)}
- Average Response Time: {df['average_time'].mean():.2f}s
- Overall Success Rate: {df['success_rate'].mean():.1f}%

## Detailed Results
"""

        for result in self.results:
            report += f"\\n### {result['agent']}\\n"
            report += f"- Average Time: {result['average_time']:.2f}s\\n"
            report += f"- Success Rate: {result['success_rate']:.1f}%\\n"
            report += f"- Iterations: {result['iterations']}\\n"

        return report
"""

with open('benchmark_script.py', 'w') as f:
    f.write(benchmark_script)

print("✅ Benchmark script created: benchmark_script.py")
print("\n⚡ Performance Optimization Summary:")
print("   • Optimization Config - Caching, database, and API settings")
print("   • Benchmark Script - Tool for measuring system performance")